## Problem Set 3 (Part 2 of 2): Interacting with the Alpha Vantage API using Python libraries

In Part 2 of Problem Set 3 we will be using the data retrived from the Alpha Vantage API and creating an optimized portfolio of stocks based on Modern Portfolio Theory. The PyPortfolioOpt package will be used to optimize a portfolio of stocks based maximizing the Sharpe Ratio. This exercise is an example of how a robo-advisor would use data APIs to continuously monitor and adjusts stock portfolios. 

This part of the exercise demonstrates:

1. Setting up a programming environment to work with APIs
2. Data preprocessing to create compatible datasets
3. Using a quantititive popular portfolio optimization library to estimate risks and weights

Documentation links:
<br />Alpha Vantage: https://www.alphavantage.co/documentation/
<br />Alpha Vantage Python library: https://alpha-vantage.readthedocs.io/en/latest/#
<br />Pandas DataFrame: https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
<br />PyPortfolioOpt: https://pyportfolioopt.readthedocs.io/en/latest/index.html

In [1]:
api_key = "A8O16AND47KXCXJM"

In [3]:
import pandas as pd
import requests
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt

# PyPortfolioOpt libraries
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [4]:
ts = TimeSeries(key=api_key,output_format='pandas')

In [5]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [8]:
print(msft_data)

             1. open   2. high    3. low  4. close   5. volume
date                                                          
1999-09-27   92.0000   92.9380   90.8750    91.438  34132200.0
1999-09-28   91.2500   92.6880   89.0661    92.125  46000600.0
1999-09-29   91.5630   92.1250   89.1250    89.500  37484400.0
1999-09-30   90.0000   91.6880   88.8130    90.563  47455400.0
1999-10-01   90.1880   90.6250   88.3123    89.984  44239000.0
1999-10-04   90.5000   92.6250   90.2500    92.563  29998000.0
1999-10-05   92.7500   93.8750   89.5000    91.813  43397000.0
1999-10-06   92.3130   94.0000   92.0630    93.688  35500200.0
1999-10-07   93.6880   95.0630   92.6880    93.750  46036400.0
1999-10-08   93.5000   95.1880   92.1250    94.938  35697000.0
1999-10-11   94.6226   95.0000   94.1250    94.313  19943800.0
1999-10-12   94.0000   94.3130   92.3750    92.563  27668600.0
1999-10-13   92.0000   93.1250   90.3130    91.063  38098400.0
1999-10-14   90.8750   92.2340   89.6880    90.688  427

Isolate "close" column of data for each security:

In [10]:
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']
#testing testing
print(fb_clean)

date
2012-05-18     38.232
2012-05-21     34.030
2012-05-22     31.000
2012-05-23     32.000
2012-05-24     33.030
2012-05-25     31.910
2012-05-29     28.840
2012-05-30     28.190
2012-05-31     29.600
2012-06-01     27.720
2012-06-04     26.900
2012-06-05     25.869
2012-06-06     26.810
2012-06-07     26.310
2012-06-08     27.100
2012-06-11     27.005
2012-06-12     27.400
2012-06-13     27.270
2012-06-14     28.290
2012-06-15     30.014
2012-06-18     31.410
2012-06-19     31.911
2012-06-20     31.600
2012-06-21     31.840
2012-06-22     33.050
2012-06-25     32.060
2012-06-26     33.100
2012-06-27     32.230
2012-06-28     31.360
2012-06-29     31.095
               ...   
2019-08-14    179.710
2019-08-15    182.590
2019-08-16    183.700
2019-08-19    186.170
2019-08-20    183.810
2019-08-21    183.550
2019-08-22    182.040
2019-08-23    177.750
2019-08-26    180.360
2019-08-27    181.300
2019-08-28    181.760
2019-08-29    185.570
2019-08-30    185.670
2019-09-03    182.390
2019-

Create new dataframe with tickers as columns:

In [13]:
df = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])
df

,fb,msft,aapl,amzn


Place data into new dataframe:

In [14]:
df['fb'] = fb_clean
df['msft'] = msft_clean
df['aapl'] = aapl_clean
df['amzn'] = amzn_clean
#Testing Testing
print(df)

                 fb     msft     aapl      amzn
date                                           
2012-05-18   38.232   29.270  530.380   213.850
2012-05-21   34.030   29.750  561.280   218.110
2012-05-22   31.000   29.760  556.970   215.330
2012-05-23   32.000   29.110  570.560   217.280
2012-05-24   33.030   29.070  565.320   215.240
2012-05-25   31.910   29.060  562.290   212.890
2012-05-29   28.840   29.560  572.270   214.750
2012-05-30   28.190   29.340  579.170   209.230
2012-05-31   29.600   29.190  577.730   212.910
2012-06-01   27.720   28.450  560.990   208.220
2012-06-04   26.900   28.550  564.290   214.570
2012-06-05   25.869   28.505  562.830   213.210
2012-06-06   26.810   29.350  571.460   217.640
2012-06-07   26.310   29.230  571.720   218.800
2012-06-08   27.100   29.650  580.320   218.480
2012-06-11   27.005   28.895  571.170   216.500
2012-06-12   27.400   29.290  576.160   216.420
2012-06-13   27.270   29.130  572.160   214.730
2012-06-14   28.290   29.340  571.530   

Take only the earlist half of observations:

In [15]:
#create df2, which wouldn't overwrite df and has half the observations
df2 = df[0:924]
df2

,fb,msft,aapl,amzn
date,,,,
2012-05-18,38.232,29.270,530.380,213.850
2012-05-21,34.030,29.750,561.280,218.110
2012-05-22,31.000,29.760,556.970,215.330
2012-05-23,32.000,29.110,570.560,217.280
2012-05-24,33.030,29.070,565.320,215.240
2012-05-25,31.910,29.060,562.290,212.890
2012-05-29,28.840,29.560,572.270,214.750
2012-05-30,28.190,29.340,579.170,209.230
2012-05-31,29.600,29.190,577.730,212.910


Calculate expected returns:

In [17]:
mu = expected_returns.mean_historical_return(df2)

Calculate sample covariance matrix:

In [19]:
S = risk_models.sample_cov(df2)
print(S)

            fb      msft      aapl      amzn
fb    0.183263  0.018451  0.020780  0.037626
msft  0.018451  0.057307  0.020826  0.026703
aapl  0.020780  0.020826  0.269413  0.020855
amzn  0.037626  0.026703  0.020855  0.093943


Optimize the weights to maximzie the Sharpe Ratio:

In [21]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)

Expected annual return: 28.3%
Annual volatility: 23.5%
Sharpe Ratio: 1.12


(0.28315300039734614, 0.2353086186227769, 1.1183313298830186)

In [22]:
print(weights)

{'fb': 0.23331629887166944, 'msft': 0.2728212151853775, 'aapl': 9.226560487851643e-17, 'amzn': 0.49386248594295307}


Using optimized weights, compute portfolio accolation:

In [23]:
latest_prices = get_latest_prices(df2)
da = DiscreteAllocation(weights, latest_prices, total_portfolio_value=1000000)
allocation, leftover = da.greedy_portfolio()
print(allocation)

1 out of 4 tickers were removed
{'amzn': 859, 'msft': 5404, 'fb': 2477}


To update portfolio, call the API again for latest data and run optimization model again:

In [24]:
fb_data, fb_meta_data = ts.get_daily(symbol='FB', outputsize='full')
msft_data, msft_meta_data = ts.get_daily(symbol='MSFT', outputsize='full')
aapl_data, aapl_meta_data = ts.get_daily(symbol='AAPL', outputsize='full')
amzn_data, amzn_meta_data = ts.get_daily(symbol='AMZN', outputsize='full')

In [25]:
#create new empty dataframe, df3
df3 = pd.DataFrame(columns=['fb', 'msft', 'aapl', 'amzn'])

In [26]:
#This wouldn't be necessary since we didn't save over data 
fb_clean = fb_data['4. close']
msft_clean = msft_data['4. close']
aapl_clean = aapl_data['4. close']
amzn_clean = amzn_data['4. close']

In [27]:
df3['fb'] = fb_clean
df3['msft'] = msft_clean
df3['aapl'] = aapl_clean
df3['amzn'] = amzn_clean
print(df3)

                 fb     msft     aapl      amzn
date                                           
2012-05-18   38.232   29.270  530.380   213.850
2012-05-21   34.030   29.750  561.280   218.110
2012-05-22   31.000   29.760  556.970   215.330
2012-05-23   32.000   29.110  570.560   217.280
2012-05-24   33.030   29.070  565.320   215.240
2012-05-25   31.910   29.060  562.290   212.890
2012-05-29   28.840   29.560  572.270   214.750
2012-05-30   28.190   29.340  579.170   209.230
2012-05-31   29.600   29.190  577.730   212.910
2012-06-01   27.720   28.450  560.990   208.220
2012-06-04   26.900   28.550  564.290   214.570
2012-06-05   25.869   28.505  562.830   213.210
2012-06-06   26.810   29.350  571.460   217.640
2012-06-07   26.310   29.230  571.720   218.800
2012-06-08   27.100   29.650  580.320   218.480
2012-06-11   27.005   28.895  571.170   216.500
2012-06-12   27.400   29.290  576.160   216.420
2012-06-13   27.270   29.130  572.160   214.730
2012-06-14   28.290   29.340  571.530   

In [28]:
mu2 = expected_returns.mean_historical_return(df3)
S2 = risk_models.sample_cov(df3)
ef2 = EfficientFrontier(mu2, S2)
weights2 = ef2.max_sharpe()
ef2.portfolio_performance(verbose=True)
print(weights2)

Expected annual return: 28.5%
Annual volatility: 22.3%
Sharpe Ratio: 1.19
{'fb': 0.14110788084764556, 'msft': 0.42339738777063635, 'aapl': 0.0, 'amzn': 0.43549473138171824}


In [30]:
latest_prices2 = get_latest_prices(df3)
da2 = DiscreteAllocation(weights2, latest_prices2, total_portfolio_value=1000000)
allocation2, leftover2 = da2.greedy_portfolio()
print(allocation2)
print(leftover2)
#How much more is left over?
print(leftover)

1 out of 4 tickers were removed
{'amzn': 246, 'msft': 3038, 'fb': 772}
493.5399999999965
29.580000000064047
